# 千帆 OpenAI 适配器

对于已经适配 OpenAI 而尚未支持千帆 SDK 的用户，本文提供了一种方法，支持用户0成本从OpenAI迁移到百度文心千帆大模型平台。
整体流程包含：
1. 在本地使用 qianfan 启动千帆OpenAI适配器
2. 编写 OpenAI规范 接口代码，通过调用本地千帆OpenAI适配器，请求千帆大模型平台。

In [ ]:
!pip install 'qianfan[openai]' -U

# 启动千帆 OpenAI 适配器

可以通过配置模型映射，将模型名称从 OpenAI 的模型映射至千帆模型:
```yaml
gpt-4.*: ERNIE-4.0-8K
```
具体示例可参考[openai_conf.yaml](./assets/openai_conf.yaml)：

In [1]:
# import nest_asyncio
import os
import subprocess as sp

os.environ["OPENAI_API_KEY"] = "FAKE_KEY"
os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8001/v1"  # 修改为本地 OpenAI 适配器地址

# 以下环境变量供千帆 OpenAI Adapter：
# 使用安全认证的access_key/secret_key:
os.environ["QIANFAN_ACCESS_KEY"] = "your_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_secret_key"
# 或者应用ak/sk
# os.environ["QIANFAN_AK"]="your_app_ak"
# os.environ["QIANFAN_SK"]="your_app_sk"

# 启动千帆 SDK 的 OpenAI Adapter 服务，用于代理转发指向 OpenAI 的请求，并替换为千帆服务的返回
server = sp.Popen("qianfan openai -p 8001 --config-file ./assets/openai_conf.yaml --no-ignore-system", shell=True)
# 也可以在终端直接执行qianfan openai -p 8001 --config-file assets/openai_conf.yaml --no-ignore-system

OpenAI wrapper server is running at                                             

 • http://127.0.0.1:8001                                                        
 • http://172.18.167.145:8001                                                   

Remember to set the environment variables:                                      

                                                                                
     export OPENAI_API_KEY='any-content-you-want'                               
     export OPENAI_BASE_URL='http://172.18.167.145:8001/v1'                     
                                                                                



INFO:     Started server process [66674]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


# 调用ERNIE文心大模型

In [6]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4", # model="ERNIE-4.0-8K",
    messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
              {'role': 'user', 'content': '如何做炒西红柿鸡蛋？'}]
)
print(completion.choices[0].message)

INFO:     127.0.0.1:63556 - "POST /v1/chat/completions HTTP/1.1" 200 OK
ChatCompletionMessage(content='炒西红柿鸡蛋的第一步就是准备好食材，包括：西红柿、鸡蛋、盐、油、葱花等。接着，打散鸡蛋，并加入少许盐搅拌均匀。然后，西红柿切块备用。在热锅中加油，油热后倒入鸡蛋液，翻炒至刚刚凝固就盛出备用。再次加油，放入葱花爆香，然后放入西红柿翻炒，炒到西红柿出汁后，放入炒好的鸡蛋，再加盐调味，翻炒均匀后即可出锅。这样就可以得到一盘即好看又好吃的西红柿炒鸡蛋了。', role='assistant', function_call=None, tool_calls=None)


# 千帆平台常用模型清单
ERNIE-3.5-8K<br>
ERNIE-4.0-8K<br>
ERNIE-Speed-8K<br>
ERNIE-Lite-8K<br>

# 查看完整模型清单

In [7]:
from qianfan import ChatCompletion
ChatCompletion().models()

{'AquilaChat-7B',
 'BLOOMZ-7B',
 'ChatGLM2-6B-32K',
 'ChatLaw',
 'EB-turbo-AppBuilder',
 'ERNIE 3.5',
 'ERNIE Speed',
 'ERNIE Speed-AppBuilder',
 'ERNIE-3.5-128K',
 'ERNIE-3.5-128K（预览版）',
 'ERNIE-3.5-8K',
 'ERNIE-3.5-8K-0205',
 'ERNIE-3.5-8K-0329',
 'ERNIE-3.5-8K-0613',
 'ERNIE-3.5-8K-Preview',
 'ERNIE-3.5-8K-preview',
 'ERNIE-3.5-preview',
 'ERNIE-4.0-8K',
 'ERNIE-4.0-8K-0104',
 'ERNIE-4.0-8K-0329',
 'ERNIE-4.0-8K-0613',
 'ERNIE-4.0-8K-Latest',
 'ERNIE-4.0-8K-Preview',
 'ERNIE-4.0-8K-Preview-0518',
 'ERNIE-4.0-8K-preview',
 'ERNIE-4.0-preview',
 'ERNIE-Bot',
 'ERNIE-Bot-4',
 'ERNIE-Bot-turbo',
 'ERNIE-Bot-turbo-AI',
 'ERNIE-Character-8K',
 'ERNIE-Character-Fiction-8K',
 'ERNIE-Function-8K',
 'ERNIE-Functions-8K',
 'ERNIE-Lite-8K',
 'ERNIE-Lite-8K-0308',
 'ERNIE-Lite-8K-0922',
 'ERNIE-Lite-8K-0922（原ERNIE-Bot-turbo-0922）',
 'ERNIE-Speed',
 'ERNIE-Speed-128K',
 'ERNIE-Speed-8K',
 'ERNIE-Tiny-8K',
 'Gemma-7B-It',
 'Gemma-7B-it',
 'Llama-2-13B-Chat',
 'Llama-2-70B-Chat',
 'Llama-2-7B-Chat'